# GPT2 with LoRA implementation in PyTorch



In [1]:
# Importing the necessary libraries

import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoConfig
from transformers import AutoTokenizer
import os
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
import pandas as pd
import torch.nn.utils.parametrize as parametrize

import random
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup
import datetime

import time

Load the GPT-2 model.

In [2]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>',fast_tokenizer=True)
config = AutoConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config,
).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
original_model = model
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:
# selecting the target modules, which in our case is attention layers.
target_names = []
for name, module in model.named_modules():
    if "attn.c_attn" in name:
        target_names.append(module)


In [ ]:
# Print the size of the weights matrices of the network
# Save the count of the total number of parameters
total_parameters_original = 0
for index, layer in enumerate(target_names):
    total_parameters_original += layer.weight.nelement() + layer.bias.nelement()
    print(f'Layer {index+1}: W: {layer.weight.shape} + B: {layer.bias.shape}')
print(f'Total number of parameters: {total_parameters_original:,}')

Layer 1: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Layer 2: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Layer 3: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Layer 4: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Layer 5: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Layer 6: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Layer 7: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Layer 8: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Layer 9: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Layer 10: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Layer 11: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Layer 12: W: torch.Size([768, 2304]) + B: torch.Size([2304])
Total number of parameters: 21,261,312


Defining the LoRA parametrization, this will force model to call the updated forward method mentioned below

In [ ]:
class LoRAParametrization(nn.Module):
    def __init__(self, features_in, features_out, rank=1, alpha=1, device='cpu'):
        super().__init__()
        # Section 4.1 of the paper:
        #   We use a random Gaussian initialization for A and zero for B, so ∆W = BA is zero at the beginning of training
        self.lora_A = nn.Parameter(torch.zeros((rank,features_out)).to(device))
        self.lora_B = nn.Parameter(torch.zeros((features_in, rank)).to(device))
        nn.init.normal_(self.lora_A, mean=0, std=1)

        # Section 4.1 of the paper:
        #   We then scale ∆Wx by α/r , where α is a constant in r.
        #   When optimizing with Adam, tuning α is roughly the same as tuning the learning rate if we scale the initialization appropriately.
        #   As a result, we simply set α to the first r we try and do not tune it.
        #   This scaling helps to reduce the need to retune hyperparameters when we vary r.
        self.scale = alpha / rank
        self.enabled = True

    def forward(self, original_weights):
        if self.enabled:
            # Return W + (B*A)*scale
            return original_weights + torch.matmul(self.lora_B, self.lora_A).view(original_weights.shape) * self.scale
        else:
            return original_weights

In [ ]:
target_names

[Conv1D(),
 Conv1D(),
 Conv1D(),
 Conv1D(),
 Conv1D(),
 Conv1D(),
 Conv1D(),
 Conv1D(),
 Conv1D(),
 Conv1D(),
 Conv1D(),
 Conv1D()]

In [ ]:
# target_modules = [model.transformer.h[0].attn.c_attn,
#  model.transformer.h[1].attn.c_attn,
#  model.transformer.h[2].attn.c_attn,
#  model.transformer.h[3].attn.c_attn,
#  model.transformer.h[4].attn.c_attn,
#  model.transformer.h[5].attn.c_attn,
#  model.transformer.h[6].attn.c_attn,
#  model.transformer.h[7].attn.c_attn,
#  model.transformer.h[8].attn.c_attn,
#  model.transformer.h[9].attn.c_attn,
#  model.transformer.h[10].attn.c_attn,
#  model.transformer.h[11].attn.c_attn]

In [ ]:
# here we register the parameterization to the target modules.

def linear_layer_parameterization(layer, device, rank=1, lora_alpha=1):
    # Only add the parameterization to the weight matrix, ignore the Bias

    # From section 4.2 of the paper:
    #   We limit our study to only adapting the attention weights for downstream tasks and freeze the MLP modules (so they are not trained in downstream tasks) both for simplicity and parameter-efficiency.
    #   [...]
    #   We leave the empirical investigation of [...], and biases to a future work.

    features_in, features_out = layer.weight.shape
    return LoRAParametrization(
        features_in, features_out, rank=rank, alpha=lora_alpha, device=device
    )

for target_module in target_names:
    parametrize.register_parametrization(
        target_module, "weight", linear_layer_parameterization(target_module, device)
    )



def enable_disable_lora(enabled=True):
    for layer in target_names:
        layer.parametrizations["weight"][0].enabled = enabled

In [ ]:
total_parameters_lora = 0
total_parameters_non_lora = 0
for index, layer in enumerate(target_names):
    total_parameters_lora += layer.parametrizations["weight"][0].lora_A.nelement() + layer.parametrizations["weight"][0].lora_B.nelement()
    total_parameters_non_lora += layer.weight.nelement() + layer.bias.nelement()
    print(
        f'Layer {index+1}: W: {layer.weight.shape} + B: {layer.bias.shape} + Lora_A: {layer.parametrizations["weight"][0].lora_A.shape} + Lora_B: {layer.parametrizations["weight"][0].lora_B.shape}'
    )
# The non-LoRA parameters count must match the original network
assert total_parameters_non_lora == total_parameters_original
print(f'Total number of parameters (original): {total_parameters_non_lora:,}')
print(f'Total number of parameters (original + LoRA): {total_parameters_lora + total_parameters_non_lora:,}')
print(f'Parameters introduced by LoRA: {total_parameters_lora:,}')
parameters_incremment = (total_parameters_lora / total_parameters_non_lora) * 100
print(f'Parameters incremment: {parameters_incremment:.3f}%')

Layer 1: W: torch.Size([768, 2304]) + B: torch.Size([2304]) + Lora_A: torch.Size([1, 2304]) + Lora_B: torch.Size([768, 1])
Layer 2: W: torch.Size([768, 2304]) + B: torch.Size([2304]) + Lora_A: torch.Size([1, 2304]) + Lora_B: torch.Size([768, 1])
Layer 3: W: torch.Size([768, 2304]) + B: torch.Size([2304]) + Lora_A: torch.Size([1, 2304]) + Lora_B: torch.Size([768, 1])
Layer 4: W: torch.Size([768, 2304]) + B: torch.Size([2304]) + Lora_A: torch.Size([1, 2304]) + Lora_B: torch.Size([768, 1])
Layer 5: W: torch.Size([768, 2304]) + B: torch.Size([2304]) + Lora_A: torch.Size([1, 2304]) + Lora_B: torch.Size([768, 1])
Layer 6: W: torch.Size([768, 2304]) + B: torch.Size([2304]) + Lora_A: torch.Size([1, 2304]) + Lora_B: torch.Size([768, 1])
Layer 7: W: torch.Size([768, 2304]) + B: torch.Size([2304]) + Lora_A: torch.Size([1, 2304]) + Lora_B: torch.Size([768, 1])
Layer 8: W: torch.Size([768, 2304]) + B: torch.Size([2304]) + Lora_A: torch.Size([1, 2304]) + Lora_B: torch.Size([768, 1])
Layer 9: W: torc

In [ ]:
# Freeze the non-Lora parameters
for name, param in model.named_parameters():
    # print(name)
    if 'lora' not in name:
        print(f'Freezing non-LoRA parameter {name}')
        param.requires_grad = False

Freezing non-LoRA parameter transformer.wte.weight
Freezing non-LoRA parameter transformer.wpe.weight
Freezing non-LoRA parameter transformer.h.0.ln_1.weight
Freezing non-LoRA parameter transformer.h.0.ln_1.bias
Freezing non-LoRA parameter transformer.h.0.attn.c_attn.bias
Freezing non-LoRA parameter transformer.h.0.attn.c_attn.parametrizations.weight.original
Freezing non-LoRA parameter transformer.h.0.attn.c_proj.weight
Freezing non-LoRA parameter transformer.h.0.attn.c_proj.bias
Freezing non-LoRA parameter transformer.h.0.ln_2.weight
Freezing non-LoRA parameter transformer.h.0.ln_2.bias
Freezing non-LoRA parameter transformer.h.0.mlp.c_fc.weight
Freezing non-LoRA parameter transformer.h.0.mlp.c_fc.bias
Freezing non-LoRA parameter transformer.h.0.mlp.c_proj.weight
Freezing non-LoRA parameter transformer.h.0.mlp.c_proj.bias
Freezing non-LoRA parameter transformer.h.1.ln_1.weight
Freezing non-LoRA parameter transformer.h.1.ln_1.bias
Freezing non-LoRA parameter transformer.h.1.attn.c_att

Training the GPT-2 model with LoRA.

In [ ]:
# Creating a dataset class for the GPT2 model.

class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
# loading the shakespeare dataset.
df = pd.read_csv('/content/drive/MyDrive/transformer/GPT/shakespeare.csv')
train_dataset = GPT2Dataset(df['text'], tokenizer, max_length=768)

In [ ]:
# creating a train dataloader.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = 4 # Trains with this batch size.
        )

In [ ]:
# this step is necessary because we have added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))


In [ ]:
# have a look at the parametrized model architecture.
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): ParametrizedConv1D(
            (parametrizations): ModuleDict(
              (weight): ParametrizationList(
                (0): LoRAParametrization()
              )
            )
          )
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  

In [ ]:
# setting up the parameters for the training loop.

epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )
# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device) # labels are shifted by the model itself in huggingface, should be the same as input_ids.
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(  b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,
                                    top_k=50,
                                    max_length = 200,
                                    top_p=0.95,
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Training Time': training_time,
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   100  of  1,806. Loss: 1.3264211416244507.   Elapsed: 0:01:14.
0:  bipartisan


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   200  of  1,806. Loss: 0.6727623343467712.   Elapsed: 0:02:32.
0:  increasing


...

  Batch 1,600  of  1,806. Loss: 0.27731460332870483.   Elapsed: 0:20:36.
0:  rents-and-mine. The question to be resolved thus is:—How much longer do you live and why do you so little to love?
It will not be to your good-pleasure. You are far worse at it, in an equal degree; and it will not be to your good-ple to be your good. The present state will come upon you, I doubt.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 1,700  of  1,806. Loss: 0.42109793424606323.   Elapsed: 0:21:54.
0:  Reg, of what shall I call them? I have no power to call these names, though the Prince was in the palace. So came my father. And so I sent my mother to marry him. And she was a great girl of her age, which was not to be known either to her husband. To be not so, indeed, I think, when she fell.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 1,800  of  1,806. Loss: 0.2709355652332306.   Elapsed: 0:23:12.
0: olas with a man so foolish

I will lie with him; for the rest is waiting
 about to strike,

  Average training loss: 0.28
  Training epoch took: 0:23:16

Training complete!
Total training took 1:58:08 (h:mm:ss)


In [ ]:
output_dir = '/content/drive/MyDrive/transformer/LoRA'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to /content/drive/MyDrive/transformer/LoRA


('/content/drive/MyDrive/transformer/LoRA/tokenizer_config.json',
 '/content/drive/MyDrive/transformer/LoRA/special_tokens_map.json',
 '/content/drive/MyDrive/transformer/LoRA/vocab.json',
 '/content/drive/MyDrive/transformer/LoRA/merges.txt',
 '/content/drive/MyDrive/transformer/LoRA/added_tokens.json',
 '/content/drive/MyDrive/transformer/LoRA/tokenizer.json')

Testing the performance of the GPT2 and LoRA tuned GPT2.

In [14]:
# testing the original GPT2 model.

model.eval()

prompt = "You will not be forgiven"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

print(generated)

sample_outputs = original_model.generate(
                                generated,
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,
                                top_k=50,
                                max_length = 300,
                                top_p=0.95,
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[ 1639,   481,   407,   307, 32350]], device='cuda:0')
0: You will not be forgiven for saying this for your own good, but I will never apologize for saying this for my own bad behavior," he said.

Bennett's campaign didn't immediately return emails requesting comment.


1: You will not be forgiven."

That's what it was about that made her leave. It was just that the moment she did leave, it was like her last chance of leaving a community in the name of what it was, as a mom, as an adult. A small part of the reason she stayed was because she wanted something. And that was that she wanted to leave the community as quickly as possible.

Now, on the other hand, it was all a waste. It was all a waste of resources. The best and hardest thing she could do now was find somewhere where she would go, somewhere that was safe, where she could take her life for the good of the community she was looking for. And with this decision to come on board a new coach at our school she had the chance

In [ ]:
# testing the LoRA finetuned model.

model.eval()

prompt = "LORD: "

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

print(generated)

sample_outputs = model.generate(
                                generated,
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,
                                top_k=50,
                                max_length = 300,
                                top_p=0.95,
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[   43, 12532,    25,   220]], device='cuda:0')
0: LORD: 탏
WILD THINGS:
I do not speak of my father.


1: LORD: -------------- I am a judge

I am, and you cannot be

I shall not bring your kingdom

into its sight.

Petitioning, please be not to

send me.


2: LORD: 不, my father: not a word is spoken in the kingdom of God, and of the Son, and of the Virgin, and of the Son's name; and
the most wonderful and great.




In [ ]:
# testing the LoRA finetuned model.

model.eval()

prompt = "You will not be forgiven"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

print(generated)

sample_outputs = model.generate(
                                generated,
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,
                                top_k=50,
                                max_length = 300,
                                top_p=0.95,
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[ 1639,   481,   407,   307, 32350]], device='cuda:0')
0: You will not be forgiven the bloodiest of your hearts, and that is my sorrow that befell you, and yet it was a part of that dreadful night when you were slain.


1: You will not be forgiven with your evil passions, with your own sin, even so long as the Lord thy God hath commanded me.

Lord, he who is now angry, yet will not endure it, that I shall lie before thee: and I shall not mourn; for I have sinned.
: And now thy last stand is upon thee, O God; for I will have an end to thy life.


2: You will not be forgiven, or you will not be forgiven' - from the Arabic word "an-dala" (saad) or "dala", translated as "she is" or "she is not."


